In [1]:
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from matplotlib import pyplot as plt
from functools import partial
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Lambda
from tensorflow.keras import regularizers
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import cv2
import os
import keras

In [2]:
df_X = pd.read_csv('dataset/train_features.csv')
df_y = pd.read_csv('dataset/train_target.csv')

In [3]:
df_X

,id,Time,S1,S2,S3,S4
0,0,0.000000,0.00,0.0,0.00,0.00
1,0,0.000004,0.00,0.0,0.00,0.00
2,0,0.000008,0.00,0.0,0.00,0.00
3,0,0.000012,0.00,0.0,0.00,0.00
4,0,0.000016,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...
1049995,2799,0.001480,-89256.32,-164614.6,48186.29,-147910.20
1049996,2799,0.001484,-29233.44,-179404.2,109534.40,-111591.10
1049997,2799,0.001488,37147.25,-190011.7,178781.10,-86836.78
1049998,2799,0.001492,98278.50,-190771.4,245726.70,-75660.00


In [4]:
df_y

,id,X,Y,M,V
0,0,0.0,-400.0,50.0,0.4
1,1,400.0,0.0,100.0,1.0
2,2,-300.0,-200.0,25.0,0.4
3,3,200.0,-100.0,150.0,0.4
4,4,-300.0,-100.0,150.0,0.4
...,...,...,...,...,...
2795,2795,200.0,200.0,50.0,1.0
2796,2796,-400.0,-400.0,150.0,0.2
2797,2797,-100.0,0.0,50.0,0.2
2798,2798,100.0,100.0,125.0,0.6


In [5]:
for col in df_X.columns:
    print(f'{col}: {df_X[col].isnull().values.any()}')
for col in df_y.columns:
    print(f'{col}: {df_y[col].isnull().values.any()}')

id: False
Time: False
S1: False
S2: False
S3: False
S4: False
id: False
X: False
Y: False
M: False
V: False


In [1]:
def data_processing(data_frame,feature_list,conc_list=list()):
    for feature in feature_list:
        conc_list.append(np.asarray(data_frame[feature]).reshape(-1,375,1))
    conc_array = np.concatenate(conc_list,axis=2)
    array = conc_array.transpose(0,2,1)
    return array

In [2]:
data = data_processing(df_X,['S1','S2','S3','S4'])

plt.figure(figsize=(6,4))
plt.plot(data[777].T)
plt.legend(['S1','S2','S3','S4'])
plt.grid(True)
plt.xlabel('time (unit)')
plt.ylabel('acceleration (mm/s^2)')

x = data
print(x.shape)

NameError: name 'df_X' is not defined

In [8]:
try_data = x[0]
N = 375 # Sample number
f = np.linspace(0,N-1,N) # Frequency step
instance_list = list()
for instance in try_data:
    y = np.array(instance).astype(np.float)
    mean = np.mean(y)
    X = np.fft.fft(y-mean)
    X_mag = np.abs(X)
    f_plot = f[0:int(N/2)]
    X_mag_plot = 2*X_mag[0:int(N/2)]
    X_mag_plot[0] /= 2
    instance_list.append(X_mag_plot)

In [9]:
fft_data_list = list()
for data_instance in data:
    N = 375 # Sample number
    f = np.linspace(0,N-1,N) # Frequency step
    instance_list = list()
    for instance in try_data:
        y = np.array(instance).astype(np.float)
        mean = np.mean(y)
        X = np.fft.fft(y-mean)
        X_mag = np.abs(X)
        f_plot = f[0:int(N/2)]
        X_mag_plot = 2*X_mag[0:int(N/2)]
        X_mag_plot[0] /= 2
        instance_list.append(X_mag_plot)
    fft_data_list.append(instance_list)

In [10]:
y = df_y.drop(['id','X','Y','V'],axis=1)
y_numpy = y.to_numpy()
y_list = y_numpy.tolist()

In [11]:
# raw waveform data
x_array = data
y_array = y_list
train_test_index = int(len(x_array)*0.9)
zip_list = list(zip(x_array,y_array))
random.shuffle(zip_list)
x_array,y_array = zip(*zip_list)
x_train,y_train = np.array(x_array[0:train_test_index]),np.array(y_array[0:train_test_index])
x_test,y_test = np.array(x_array[train_test_index:]),np.array(y_array[train_test_index:])

In [12]:
# fast fourier transform data
x_array_fft = fft_data_list
y_array_fft = y_list
train_test_index_fft = int(len(x_array_fft)*0.9)
zip_list_fft = list(zip(x_array_fft,y_array_fft))
random.shuffle(zip_list_fft)
x_array_fft,y_array_fft = zip(*zip_list_fft)
x_train_fft,y_train_fft = np.array(x_array_fft[0:train_test_index_fft]),np.array(y_array_fft[0:train_test_index_fft])
x_test_fft,y_test_fft = np.array(x_array_fft[train_test_index_fft:]),np.array(y_array_fft[train_test_index_fft:])

In [13]:
x_train.shape

(2520, 4, 375)

In [67]:
x_train[0][0][0:200].shape

(200,)

In [110]:
from sklearn.preprocessing import StandardScaler
patch_number = 4
# raw waveform data
for patch in range(patch_number):
    vars()[f'train_list{patch+1}'] = list()
    vars()[f'test_list{patch+1}'] = list()

for ele in x_train:
    scaler = StandardScaler()
    ele = scaler.fit_transform(ele)
    train_list1.append(ele[0])
    train_list2.append(ele[1])
    train_list3.append(ele[2])
    train_list4.append(ele[3])

for ele in x_test:
    scaler = StandardScaler()
    ele = scaler.fit_transform(ele)
    test_list1.append(ele[0])
    test_list2.append(ele[0])
    test_list3.append(ele[0])
    test_list4.append(ele[0])
    
# fast fourier transform
for patch in range(patch_number):
    vars()[f'train_list_fft{patch+1}'] = list()
    vars()[f'test_list_fft{patch+1}'] = list()

for ele in x_train_fft:
    train_list_fft1.append(ele[0])
    train_list_fft2.append(ele[1])
    train_list_fft3.append(ele[2])
    train_list_fft4.append(ele[3])

for ele in x_test_fft:
    test_list_fft1.append(ele[0])
    test_list_fft2.append(ele[1])
    test_list_fft3.append(ele[2])
    test_list_fft4.append(ele[3])

# for Global training and testing
train_fit = list()
train_fft_fit = list()
test_fit = list()
test_fft_fit = list()

for i in range(4):
    vars()[f'train_list{i+1}'] = np.array(vars()[f'train_list{i+1}'])
    train_fit.append(vars()[f'train_list{i+1}'])
    vars()[f'train_list_fft{i+1}'] = np.array(vars()[f'train_list_fft{i+1}'])
    train_fft_fit.append(vars()[f'train_list_fft{i+1}'])
    vars()[f'test_list{i+1}'] = np.array(vars()[f'test_list{i+1}'])
    test_fit.append(vars()[f'test_list{i+1}'])
    vars()[f'test_list_fft{i+1}'] = np.array(vars()[f'test_list_fft{i+1}'])
    test_fft_fit.append(vars()[f'test_list_fft{i+1}'])
    

# for PatchNet training
train_list1 = np.expand_dims(train_list1,axis=2)
train_list2 = np.expand_dims(train_list2,axis=2)
train_list3 = np.expand_dims(train_list3,axis=2)
train_list4 = np.expand_dims(train_list4,axis=2)

train_list_fft1 = np.expand_dims(train_list_fft1,axis=2)
train_list_fft2 = np.expand_dims(train_list_fft2,axis=2)
train_list_fft3 = np.expand_dims(train_list_fft3,axis=2)
train_list_fft4 = np.expand_dims(train_list_fft4,axis=2)

In [111]:
from sklearn.preprocessing import MinMaxScaler

In [112]:
Scaler = MinMaxScaler(feature_range=(0, 1))

In [113]:
y_train_scaled = Scaler.fit_transform(y_train)

In [114]:
y_train_scaled

array([[1.        ],
       [1.        ],
       [0.66666667],
       ...,
       [0.83333333],
       [0.5       ],
       [0.5       ]])

In [115]:
def PatchNet_HP(hp):
    #scaler = hp.Int('scaler',min_value=5,max_value=10,step=1)
    defaultConv1D = partial(tf.keras.layers.Conv1D,kernel_size=3,
                                                   kernel_initializer=tf.keras.initializers.lecun_normal)
    momentums=hp.Choice('momentum',[0.9,0.99,0.999])
    learning_rate = hp.Choice('learning_rate',[0.1,0.01,0.001])
    #learning_rate = 0.001
    
    model = tf.keras.Sequential()
    model.add(defaultConv1D(filters=64))
    model.add(tf.keras.layers.BatchNormalization(momentum=momentums))
    model.add(tf.keras.layers.Activation('selu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(defaultConv1D(filters=64))
    model.add(tf.keras.layers.BatchNormalization(momentum=momentums))
    model.add(tf.keras.layers.Activation('selu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(defaultConv1D(filters=64))
    model.add(tf.keras.layers.BatchNormalization(momentum=momentums))
    model.add(tf.keras.layers.Activation('selu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(defaultConv1D(filters=64))
    model.add(tf.keras.layers.BatchNormalization(momentum=momentums))
    model.add(tf.keras.layers.Activation('selu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=50,kernel_initializer=tf.keras.initializers.lecun_normal))
    model.add(tf.keras.layers.BatchNormalization(momentum=momentums))
    model.add(tf.keras.layers.Activation('selu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(units=1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="mean_absolute_error",
                  metrics=["mean_absolute_error"])
    return model

In [116]:
tuner1 = BayesianOptimization(PatchNet_HP,
                              objective='val_mean_absolute_error',
                              max_trials=20,
                              directory='D:/Users/Admin/Desktop/raw/weights')

In [117]:
tuner1.search(train_list1,y_train_scaled,batch_size=16,epochs=20,validation_split=0.2)

Trial 6 Complete [00h 01m 06s]
val_mean_absolute_error: 0.15475837886333466

Best val_mean_absolute_error So Far: 0.15468791127204895
Total elapsed time: 00h 06m 47s

Search: Running Trial #7

Hyperparameter    |Value             |Best Value So Far 
momentum          |0.999             |0.9               
learning_rate     |0.001             |0.01              

Epoch 1/20
126/126 [==============================] - 5s 30ms/step - loss: 1.5551 - mean_absolute_error: 1.5551 - val_loss: 2.9557 - val_mean_absolute_error: 2.9557
Epoch 2/20
126/126 [==============================] - 3s 23ms/step - loss: 0.9327 - mean_absolute_error: 0.9327 - val_loss: 2.2096 - val_mean_absolute_error: 2.2096
Epoch 3/20
126/126 [==============================] - 3s 25ms/step - loss: 0.6827 - mean_absolute_error: 0.6827 - val_loss: 1.0920 - val_mean_absolute_error: 1.0920
Epoch 4/20
126/126 [==============================] - 3s 26ms/step - loss: 0.4440 - mean_absolute_error: 0.4440 - val_loss: 0.5681 - val_mea

KeyboardInterrupt: 

In [149]:
tuner1.results_summary()

Results summary
Results in D:/Users/Admin/Desktop/raw/weights\untitled_project
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
scaler: 9
momentum: 0.9
learning_rate: 0.01
Score: 21.375234603881836
Trial summary
Hyperparameters:
scaler: 9
momentum: 0.9
learning_rate: 0.1
Score: 21.47659683227539
Trial summary
Hyperparameters:
scaler: 8
momentum: 0.9
learning_rate: 0.001
Score: 21.530128479003906
Trial summary
Hyperparameters:
scaler: 10
momentum: 0.9
learning_rate: 0.001
Score: 21.684776306152344
Trial summary
Hyperparameters:
scaler: 5
momentum: 0.9
learning_rate: 0.1
Score: 21.69649887084961
Trial summary
Hyperparameters:
scaler: 9
momentum: 0.9
learning_rate: 0.01
Score: 21.803020477294922
Trial summary
Hyperparameters:
scaler: 8
momentum: 0.9
learning_rate: 0.01
Score: 21.828350067138672
Trial summary
Hyperparameters:
scaler: 9
momentum: 0.9
learning_rate: 0.1
Score: 21.877458572387695
Trial summary
Hyperparameters:
sc

In [44]:
from keras import callbacks
early_stopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

## Model 1

In [45]:
defaultConv1D = partial(tf.keras.layers.Conv1D,kernel_size=3,
                                               kernel_initializer=tf.keras.initializers.lecun_normal)
scaler = 10
momentum = 0.9
learning_rate = 0.001

model1 = tf.keras.Sequential()
model1.add(defaultConv1D(filters=2*scaler))
model1.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model1.add(tf.keras.layers.Activation('selu'))
model1.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model1.add(defaultConv1D(filters=3*scaler))
model1.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model1.add(tf.keras.layers.Activation('selu'))
model1.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model1.add(defaultConv1D(filters=4*scaler))
model1.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model1.add(tf.keras.layers.Activation('selu'))
model1.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model1.add(defaultConv1D(filters=5*scaler))
model1.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model1.add(tf.keras.layers.Activation('selu'))
model1.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(units=50,kernel_initializer=tf.keras.initializers.lecun_normal))
model1.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model1.add(tf.keras.layers.Activation('selu'))
model1.add(tf.keras.layers.Dropout(0.5))
model1.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='mean_absolute_error',
              metrics=['mean_absolute_error'])

In [46]:
history1 = model1.fit(train_list1,y_train_scaled,batch_size=32,epochs=50,
                      validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
63/63 [==============================] - 9s 68ms/step - loss: 0.2807 - mean_absolute_error: 0.2807 - val_loss: 0.1461 - val_mean_absolute_error: 0.1461
Epoch 2/50
63/63 [==============================] - 3s 47ms/step - loss: 0.1789 - mean_absolute_error: 0.1789 - val_loss: 0.1343 - val_mean_absolute_error: 0.1343
Epoch 3/50
63/63 [==============================] - 3s 44ms/step - loss: 0.1673 - mean_absolute_error: 0.1673 - val_loss: 0.1208 - val_mean_absolute_error: 0.1208
Epoch 4/50
63/63 [==============================] - 2s 26ms/step - loss: 0.1520 - mean_absolute_error: 0.1520 - val_loss: 0.1187 - val_mean_absolute_error: 0.1187
Epoch 5/50
63/63 [==============================] - 2s 29ms/step - loss: 0.1388 - mean_absolute_error: 0.1388 - val_loss: 0.1050 - val_mean_absolute_error: 0.1050
Epoch 6/50
63/63 [==============================] - 2s 34ms/step - loss: 0.1353 - mean_absolute_error: 0.1353 - val_loss: 0.1077 - val_mean_absolute_error: 0.1077
Epoch 7/50
63/63 [====

In [47]:
defaultConv1D = partial(tf.keras.layers.Conv1D,kernel_size=3,
                                               kernel_initializer=tf.keras.initializers.lecun_normal)
scaler = 10
momentum = 0.9
learning_rate = 0.001

model2 = tf.keras.Sequential()
model2.add(defaultConv1D(filters=2*scaler))
model2.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model2.add(tf.keras.layers.Activation('selu'))
model2.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model2.add(defaultConv1D(filters=3*scaler))
model2.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model2.add(tf.keras.layers.Activation('selu'))
model2.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model2.add(defaultConv1D(filters=4*scaler))
model2.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model2.add(tf.keras.layers.Activation('selu'))
model2.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model2.add(defaultConv1D(filters=5*scaler))
model2.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model2.add(tf.keras.layers.Activation('selu'))
model2.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(units=50,kernel_initializer=tf.keras.initializers.lecun_normal))
model2.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model2.add(tf.keras.layers.Activation('selu'))
model2.add(tf.keras.layers.Dropout(0.5))
model2.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='mean_absolute_error',
              metrics=['mean_absolute_error'])

In [48]:
history2 = model2.fit(train_list2,y_train_scaled,batch_size=16,epochs=25,
                      validation_split=0.2,callbacks=[early_stopping])

Epoch 1/25
126/126 [==============================] - 8s 33ms/step - loss: 0.2630 - mean_absolute_error: 0.2630 - val_loss: 0.1572 - val_mean_absolute_error: 0.1572
Epoch 2/25
126/126 [==============================] - 4s 34ms/step - loss: 0.1861 - mean_absolute_error: 0.1861 - val_loss: 0.1330 - val_mean_absolute_error: 0.1330
Epoch 3/25
126/126 [==============================] - 4s 29ms/step - loss: 0.1767 - mean_absolute_error: 0.1767 - val_loss: 0.1313 - val_mean_absolute_error: 0.1313
Epoch 4/25
126/126 [==============================] - 2s 16ms/step - loss: 0.1520 - mean_absolute_error: 0.1520 - val_loss: 0.1036 - val_mean_absolute_error: 0.1036
Epoch 5/25
126/126 [==============================] - 2s 17ms/step - loss: 0.1489 - mean_absolute_error: 0.1489 - val_loss: 0.0889 - val_mean_absolute_error: 0.0889
Epoch 6/25
126/126 [==============================] - 3s 24ms/step - loss: 0.1334 - mean_absolute_error: 0.1334 - val_loss: 0.0935 - val_mean_absolute_error: 0.0935
Epoch 7/25

In [49]:
defaultConv1D = partial(tf.keras.layers.Conv1D,kernel_size=3,
                                               kernel_initializer=tf.keras.initializers.lecun_normal)
scaler = 10
momentum = 0.9
learning_rate = 0.001

model3 = tf.keras.Sequential()
model3.add(defaultConv1D(filters=2*scaler))
model3.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model3.add(tf.keras.layers.Activation('selu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model3.add(defaultConv1D(filters=3*scaler))
model3.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model3.add(tf.keras.layers.Activation('selu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model3.add(defaultConv1D(filters=4*scaler))
model3.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model3.add(tf.keras.layers.Activation('selu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model3.add(defaultConv1D(filters=5*scaler))
model3.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model3.add(tf.keras.layers.Activation('selu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(units=50,kernel_initializer=tf.keras.initializers.lecun_normal))
model3.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model3.add(tf.keras.layers.Activation('selu'))
model3.add(tf.keras.layers.Dropout(0.5))
model3.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='mean_absolute_error',
              metrics=['mean_absolute_error'])

In [50]:
history3 = model3.fit(train_list3,y_train_scaled,batch_size=16,epochs=25,
                      validation_split=0.2,callbacks=[early_stopping])

Epoch 1/25
126/126 [==============================] - 9s 21ms/step - loss: 0.2539 - mean_absolute_error: 0.2539 - val_loss: 0.1405 - val_mean_absolute_error: 0.1405
Epoch 2/25
126/126 [==============================] - 2s 20ms/step - loss: 0.1763 - mean_absolute_error: 0.1763 - val_loss: 0.1275 - val_mean_absolute_error: 0.1275
Epoch 3/25
126/126 [==============================] - 3s 21ms/step - loss: 0.1599 - mean_absolute_error: 0.1599 - val_loss: 0.1163 - val_mean_absolute_error: 0.1163
Epoch 4/25
126/126 [==============================] - 3s 24ms/step - loss: 0.1581 - mean_absolute_error: 0.1581 - val_loss: 0.1090 - val_mean_absolute_error: 0.1090
Epoch 5/25
126/126 [==============================] - 3s 27ms/step - loss: 0.1426 - mean_absolute_error: 0.1426 - val_loss: 0.0878 - val_mean_absolute_error: 0.0878
Epoch 6/25
126/126 [==============================] - 4s 32ms/step - loss: 0.1276 - mean_absolute_error: 0.1276 - val_loss: 0.0967 - val_mean_absolute_error: 0.0967
Epoch 7/25

In [51]:
defaultConv1D = partial(tf.keras.layers.Conv1D,kernel_size=3,
                                               kernel_initializer=tf.keras.initializers.lecun_normal)
scaler = 10
momentum = 0.9
learning_rate = 0.001

model4 = tf.keras.Sequential()
model4.add(defaultConv1D(filters=2*scaler))
model4.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model4.add(tf.keras.layers.Activation('selu'))
model4.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model4.add(defaultConv1D(filters=3*scaler))
model4.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model4.add(tf.keras.layers.Activation('selu'))
model4.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model4.add(defaultConv1D(filters=4*scaler))
model4.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model4.add(tf.keras.layers.Activation('selu'))
model4.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model4.add(defaultConv1D(filters=5*scaler))
model4.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model4.add(tf.keras.layers.Activation('selu'))
model4.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model4.add(tf.keras.layers.Flatten())
model4.add(tf.keras.layers.Dense(units=50,kernel_initializer=tf.keras.initializers.lecun_normal))
model4.add(tf.keras.layers.BatchNormalization(momentum=momentum))
model4.add(tf.keras.layers.Activation('selu'))
model4.add(tf.keras.layers.Dropout(0.5))
model4.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

model4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='mean_absolute_error',
              metrics=['mean_absolute_error'])

In [52]:
history4 = model4.fit(train_list4,y_train_scaled,batch_size=16,epochs=25,
                      validation_split=0.2,callbacks=[early_stopping])

Epoch 1/25
126/126 [==============================] - 6s 27ms/step - loss: 0.2623 - mean_absolute_error: 0.2623 - val_loss: 0.1620 - val_mean_absolute_error: 0.1620
Epoch 2/25
126/126 [==============================] - 3s 24ms/step - loss: 0.1862 - mean_absolute_error: 0.1862 - val_loss: 0.1298 - val_mean_absolute_error: 0.1298
Epoch 3/25
126/126 [==============================] - 3s 22ms/step - loss: 0.1612 - mean_absolute_error: 0.1612 - val_loss: 0.1165 - val_mean_absolute_error: 0.1165
Epoch 4/25
126/126 [==============================] - 4s 31ms/step - loss: 0.1556 - mean_absolute_error: 0.1556 - val_loss: 0.1094 - val_mean_absolute_error: 0.1094
Epoch 5/25
126/126 [==============================] - 4s 28ms/step - loss: 0.1452 - mean_absolute_error: 0.1452 - val_loss: 0.0939 - val_mean_absolute_error: 0.0939
Epoch 6/25
126/126 [==============================] - 2s 19ms/step - loss: 0.1338 - mean_absolute_error: 0.1338 - val_loss: 0.1047 - val_mean_absolute_error: 0.1047
Epoch 7/25

In [53]:
momentum = 0.9
learning_rate = 0.001
conc_list = list()
input_list = list()
for i in range(4):
    model = vars()[f'model{i+1}']
    model = Model(inputs=model.input,outputs=model.layers[-6].output)
    input_list.append(model.input)
    conc_list.append(model.output)
concatenate = Concatenate()(conc_list)
dense = tf.keras.layers.Dense(units=200,kernel_initializer=tf.keras.initializers.lecun_normal)(concatenate)
batchnorm = tf.keras.layers.BatchNormalization(momentum=momentum)(dense)
activation = tf.keras.layers.Activation('selu')(batchnorm)
dropout = tf.keras.layers.Dropout(0.5)(activation)
output = tf.keras.layers.Dense(units=1,activation='sigmoid')(dropout)

final_model = Model(inputs=input_list,outputs=output)
final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss='mean_absolute_error',
                    metrics=['mean_absolute_error'])

In [54]:
history_final = final_model.fit(train_fit,y_train_scaled,validation_split=0.2,batch_size=16,epochs=25,
                callbacks=[early_stopping])

Epoch 1/25
126/126 [==============================] - 23s 66ms/step - loss: 0.2562 - mean_absolute_error: 0.2562 - val_loss: 0.2108 - val_mean_absolute_error: 0.2108
Epoch 2/25
126/126 [==============================] - 9s 69ms/step - loss: 0.1976 - mean_absolute_error: 0.1976 - val_loss: 0.1528 - val_mean_absolute_error: 0.1528
Epoch 3/25
126/126 [==============================] - 7s 55ms/step - loss: 0.1688 - mean_absolute_error: 0.1688 - val_loss: 0.0973 - val_mean_absolute_error: 0.0973
Epoch 4/25
126/126 [==============================] - 4s 35ms/step - loss: 0.1304 - mean_absolute_error: 0.1304 - val_loss: 0.0794 - val_mean_absolute_error: 0.0794
Epoch 5/25
126/126 [==============================] - 5s 38ms/step - loss: 0.1202 - mean_absolute_error: 0.1202 - val_loss: 0.0854 - val_mean_absolute_error: 0.0854
Epoch 6/25
126/126 [==============================] - 6s 48ms/step - loss: 0.1097 - mean_absolute_error: 0.1097 - val_loss: 0.0670 - val_mean_absolute_error: 0.0670
Epoch 7/2

In [60]:
y_test_scaled = Scaler.fit_transform(y_test)

In [61]:
final_model.evaluate(test_fit,y_test_scaled)

9/9 [==============================] - 0s 20ms/step - loss: 0.3734 - mean_absolute_error: 0.3734


[0.3733775317668915, 0.3733775317668915]

In [ ]:
# 1) standard scaler
# 2) convergence
# 3) 2D conv